# このノートブックについて

https://qiita.com/f-paico/items/4104bc3ba7501ef57fa6 を拝見しながら動作確認したもの。

# 電気のオン・オフ

In [5]:
import socket
import binascii

In [6]:
def light_switch(command):
    if command == 'on':
        prop = '30'
    elif command == 'off':
        prop = '31'
    else:
        prop = '31'
        
    # MoekadenRoomを実行しているPCのIPアドレス（各自変更する）
    ip = '10.0.4.71'
    # ECHONET LiteはUDPの3610番ポートを使用する
    ECHONETport = 3610
    format_echonet_lite = ['EHD', 'TID', 'SEOJ', 'DEOJ', 'ESV', 'OPC', 'EPC', 'PDC', 'EDT']
    data_format = {
        format_echonet_lite[0]: '1081',
        format_echonet_lite[1]: '0000',
        format_echonet_lite[2]: '05FF01',
        format_echonet_lite[3]: '029001',
        format_echonet_lite[4]: '60',
        format_echonet_lite[5]: '01',
        format_echonet_lite[6]: '80',
        format_echonet_lite[7]: '01',
        format_echonet_lite[8]: prop
    }

    frame = ''
    for key in format_echonet_lite:
        frame += data_format[key]

    # 16進数文字列をバイト列に変換する
    msg = binascii.unhexlify(frame)

    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.sendto(msg, (ip, ECHONETport))

In [7]:
light_switch('on')

In [8]:
light_switch('off')

# 応答を受け取る

In [1]:
import socket
import binascii
from threading import Thread
import time

In [4]:
def light_switch(command):
    if command == 'on':
        prop = '30'
    elif command == 'off':
        prop = '31'
    else:
        prop = '31'
        
    # MoekadenRoomを実行しているPCのIPアドレス（各自変更する）
    ip = '10.0.4.71'
    # ECHONET LiteはUDPの3610番ポートを使用する
    ECHONETport = 3610
    format_echonet_lite = ['EHD', 'TID', 'SEOJ', 'DEOJ', 'ESV', 'OPC', 'EPC', 'PDC', 'EDT']
    data_format = {
        format_echonet_lite[0]: '1081',
        format_echonet_lite[1]: '0000',
        format_echonet_lite[2]: '05FF01',
        format_echonet_lite[3]: '029001',
        format_echonet_lite[4]: '61',  # 応答を受け取る
        format_echonet_lite[5]: '01',
        format_echonet_lite[6]: '80',
        format_echonet_lite[7]: '01',
        format_echonet_lite[8]: prop
    }

    frame = ''
    for key in format_echonet_lite:
        frame += data_format[key]

    # 16進数文字列をバイト列に変換する
    msg = binascii.unhexlify(frame)

    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.sendto(msg, (ip, ECHONETport))
    print('send')

In [6]:
light_switch('off')

send


# マルチキャストで試す

In [15]:
# ---------------------------------------------------
# 3.2.1 ECHONET Lite ヘッダ(EHD)
# ---------------------------------------------------
# 3.2.1.1 ECHONET Lite ヘッダ 1(EHD1)
EHD1 = "10"  # ECHONET Lite規格

# 3.2.1.2 ECHONET Lite ヘッダ 2(EHD2)
EHD2 = "81"  # 形式１（規定電文形式）

# 3.2.2 Transaction ID(TID)
TID = "0001"  # IDなのでこの検証ではどの値でもOK

# フレームのヘッダ－を構成
EHD = EHD1 + EHD2 + TID

In [16]:
# ---------------------------------------------------
# 3.2.1 ECHONET Lite データ(EDATA)
# ---------------------------------------------------
# 3.2.4 ECHONETオブジェクト
# EOJ = ECHONET Lite オブジェクト

# SEOJ = 送信元ECHONET Lite オブジェクト
SEOJ_CLS_GROUP = "05"     # 管理・操作関連クラスグループ
SEOJ_CLS_CODE = "ff"      # コントローラー
SEOJ_CLS_INSTANCE = "01"  # インスタンス番号
SEOJ = SEOJ_CLS_GROUP + SEOJ_CLS_CODE + SEOJ_CLS_INSTANCE

# DEOJ = 送信先ECHONET Lite オブジェクト
DEOJ_CLS_GROUP = "01"     # 空調関連機器クラスグループ
DEOJ_CLS_CODE = "30"      # 家庭用エアコンクラス
DEOJ_CLS_INSTANCE = "01"  # All Instanses
DEOJ = DEOJ_CLS_GROUP + DEOJ_CLS_CODE + DEOJ_CLS_INSTANCE


# 3.2.5 ECHONET Lite サービス(ESV)
ESV = "62"  # プロパティ値読み出し要求 (Get)

###############
# APPENDIX ECHONET機器オブジェクト詳細規定
# - 空調関連機器クラスグループ
#  - 家庭用エアコンクラス規定
# を確認

# 3.2.6 処理プロパティカウンタ
OPC = "02"  # 2件


# プロパティ 1件目
EPC1 = "80"    # 動作状態の取得(On or Off)
PDC1 = "00"    # Getの場合は0でOK
EDT1 = ""      # Getの場合はEDTは不要
PROP1 = EPC1 + PDC1 + EDT1

# プロパティ 2件目
EPC2 = "b3"    # 温度設定の取得
PDC2 = "00"    # Getの場合は0でOK
EDT2 = ""      # Getの場合はEDTは不要
PROP2 = EPC2 + PDC2 + EDT2


# フレームのデータ部分であるEDATAを構成
EDATA = SEOJ + DEOJ + ESV + OPC + PROP1 + PROP2

In [17]:
echonet_command = EHD + EDATA


In [21]:
import socket

echonet_port = 3610
aircon_ip = "224.0.23.0"  # 224.0.23.0のアドレスを使うとマルチキャストもできます

# 応答待ち受け用ソケット
#recv_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#recv_sock.bind(("0.0.0.0", echonet_port))

# 要求送信用ソケットでコマンド送信
#send_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#send_sock.sendto(bytes.fromhex(echonet_command), (aircon_ip, echonet_port))
sock.sendto(bytes.fromhex(echonet_command), (aircon_ip, echonet_port))

NameError: name 'sock' is not defined